In [1]:
import pandas as pd
from datetime import datetime
from osgeo import gdal
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from dateutil import parser
import os

In [10]:
# train_labels = pd.read_csv("train_labels.csv")  # Full training dataset
train_labels = pd.read_csv("train_labels_10k.csv")  # Smaller subset
grid_metadata = pd.read_csv("grid_metadata.csv")
satellite_metadata = pd.read_csv("pm25_satellite_metadata.csv")
submission_format = pd.read_csv("submission_format.csv")
satellite_metadata['Date'] =  pd.to_datetime(satellite_metadata['time_end'], format='%Y-%m-%d')

In [11]:
####################
# REMOVE THIS LINE #
####################
# train_labels = train_labels.sample(2, random_state=42)

In [12]:
def get_grid_data(metadata, grid_id):
    return metadata[metadata["grid_id"] == grid_id]

In [13]:
def fetch_satellite_meta(metadata, datetime, location, datatype, split):
    if location == "Delhi":
        location = "dl"
    elif location == "Taipei":
        location = "tpe"
    else:
        location = "la"

    metadata = metadata[metadata['location'] == location]
    metadata = metadata[metadata['product'] == datatype]
    metadata = metadata[metadata['split'] == split]
    dateobject = parser.parse(datetime)
    return metadata.loc[(metadata['Date'].dt.month == dateobject.month) & 
                        (metadata['Date'].dt.day == dateobject.day) &
                        (metadata['Date'].dt.year <= dateobject.year)]

In [14]:
# Opens the HDF file
def load_data(FILEPATH):
    ds = gdal.Open(FILEPATH)
    return ds

def fetch_subset(granule_id, split="train"):
    print("Granule ID: ", granule_id)
    ds = load_data(os.path.join(split, granule_id))
    ds.GetSubDatasets()[0]
    raster = gdal.Open(ds.GetSubDatasets()[8][0]) #grid5km:cosSZA features only
    band = raster.GetRasterBand(1)
    band_arr = band.ReadAsArray()
    return band_arr

In [15]:
def fetch_training_features(grid_id, datetime, split):
    temp = get_grid_data(grid_metadata, grid_id)
    sat_met = fetch_satellite_meta(satellite_metadata, 
                               datetime, 
                               temp.iloc[0]['location'], 
                               "maiac", 
                               split)
    counter = 0
    features = None
    for i in range(len(sat_met)):
        counter += 1
        granule_id = sat_met.iloc[i]['granule_id']
        subset = fetch_subset(granule_id, split=split)
        if features is None:
            features = subset
        else:
            features += subset
    return features/counter

In [16]:
def generate_features(train_labels, split):
    labels = []
    features = []
    for i in range(len(train_labels)):
        feature = fetch_training_features(train_labels.iloc[i]['grid_id'], train_labels.iloc[i]['datetime'], split)
        features.append(np.array(feature).reshape(-1))
        if split == "train":
            labels.append(train_labels.iloc[i]['value'])
    return np.array(features), np.array(labels)

In [17]:
features, labels = generate_features(train_labels, "train")

Granule ID:  20180201T191000_maiac_la_0.hdf
Granule ID:  20180201T191000_maiac_la_0.hdf
Granule ID:  20180201T191000_maiac_la_0.hdf
Granule ID:  20180201T191000_maiac_la_0.hdf
Granule ID:  20180201T191000_maiac_la_0.hdf
Granule ID:  20180201T191000_maiac_la_0.hdf
Granule ID:  20180201T191000_maiac_la_0.hdf
Granule ID:  20180201T191000_maiac_la_0.hdf
Granule ID:  20180201T191000_maiac_la_0.hdf
Granule ID:  20180201T191000_maiac_la_0.hdf
Granule ID:  20180201T191000_maiac_la_0.hdf
Granule ID:  20180201T191000_maiac_la_0.hdf
Granule ID:  20180201T024000_maiac_tpe_0.hdf
Granule ID:  20180201T042000_maiac_tpe_0.hdf
Granule ID:  20180201T024000_maiac_tpe_0.hdf
Granule ID:  20180201T042000_maiac_tpe_0.hdf
Granule ID:  20180201T024000_maiac_tpe_0.hdf
Granule ID:  20180201T042000_maiac_tpe_0.hdf
Granule ID:  20180201T024000_maiac_tpe_0.hdf
Granule ID:  20180201T042000_maiac_tpe_0.hdf
Granule ID:  20180201T024000_maiac_tpe_0.hdf
Granule ID:  20180201T042000_maiac_tpe_0.hdf
Granule ID:  20180201T

Granule ID:  20180206T192500_maiac_la_0.hdf
Granule ID:  20180206T030000_maiac_tpe_0.hdf
Granule ID:  20180206T030000_maiac_tpe_1.hdf
Granule ID:  20180206T030000_maiac_tpe_0.hdf
Granule ID:  20180206T030000_maiac_tpe_1.hdf
Granule ID:  20180206T030000_maiac_tpe_0.hdf
Granule ID:  20180206T030000_maiac_tpe_1.hdf
Granule ID:  20180206T030000_maiac_tpe_0.hdf
Granule ID:  20180206T030000_maiac_tpe_1.hdf
Granule ID:  20180206T030000_maiac_tpe_0.hdf
Granule ID:  20180206T030000_maiac_tpe_1.hdf
Granule ID:  20180206T030000_maiac_tpe_0.hdf
Granule ID:  20180206T030000_maiac_tpe_1.hdf
Granule ID:  20180206T030000_maiac_tpe_0.hdf
Granule ID:  20180206T030000_maiac_tpe_1.hdf
Granule ID:  20180206T062000_maiac_dl_0.hdf
Granule ID:  20180206T062000_maiac_dl_0.hdf
Granule ID:  20180206T062000_maiac_dl_0.hdf
Granule ID:  20180206T062000_maiac_dl_0.hdf
Granule ID:  20180206T062000_maiac_dl_0.hdf
Granule ID:  20180206T062000_maiac_dl_0.hdf
Granule ID:  20180206T062000_maiac_dl_0.hdf
Granule ID:  20180

Granule ID:  20180211T032000_maiac_tpe_1.hdf
Granule ID:  20180211T032000_maiac_tpe_0.hdf
Granule ID:  20180211T032000_maiac_tpe_1.hdf
Granule ID:  20180211T032000_maiac_tpe_0.hdf
Granule ID:  20180211T032000_maiac_tpe_1.hdf
Granule ID:  20180211T032000_maiac_tpe_0.hdf
Granule ID:  20180211T032000_maiac_tpe_1.hdf
Granule ID:  20180211T032000_maiac_tpe_0.hdf
Granule ID:  20180211T032000_maiac_tpe_1.hdf
Granule ID:  20180211T032000_maiac_tpe_0.hdf
Granule ID:  20180211T032000_maiac_tpe_1.hdf
Granule ID:  20180211T063500_maiac_dl_0.hdf
Granule ID:  20180211T063500_maiac_dl_0.hdf
Granule ID:  20180211T063500_maiac_dl_0.hdf
Granule ID:  20180211T063500_maiac_dl_0.hdf
Granule ID:  20180211T063500_maiac_dl_0.hdf
Granule ID:  20180211T063500_maiac_dl_0.hdf
Granule ID:  20180211T063500_maiac_dl_0.hdf
Granule ID:  20180211T063500_maiac_dl_0.hdf
Granule ID:  20180211T063500_maiac_dl_0.hdf
Granule ID:  20180211T063500_maiac_dl_0.hdf
Granule ID:  20180211T063500_maiac_dl_0.hdf
Granule ID:  20180212

Granule ID:  20180216T033500_maiac_tpe_0.hdf
Granule ID:  20180216T034000_maiac_tpe_0.hdf
Granule ID:  20180216T033500_maiac_tpe_0.hdf
Granule ID:  20180216T065500_maiac_dl_0.hdf
Granule ID:  20180216T065500_maiac_dl_0.hdf
Granule ID:  20180216T065500_maiac_dl_0.hdf
Granule ID:  20180216T065500_maiac_dl_0.hdf
Granule ID:  20180216T065500_maiac_dl_0.hdf
Granule ID:  20180216T065500_maiac_dl_0.hdf
Granule ID:  20180216T065500_maiac_dl_0.hdf
Granule ID:  20180216T065500_maiac_dl_0.hdf
Granule ID:  20180216T065500_maiac_dl_0.hdf
Granule ID:  20180216T065500_maiac_dl_0.hdf
Granule ID:  20180216T065500_maiac_dl_0.hdf
Granule ID:  20180216T065500_maiac_dl_0.hdf
Granule ID:  20180217T191000_maiac_la_0.hdf
Granule ID:  20180217T191000_maiac_la_0.hdf
Granule ID:  20180217T191000_maiac_la_0.hdf
Granule ID:  20180217T191000_maiac_la_0.hdf
Granule ID:  20180217T191000_maiac_la_0.hdf
Granule ID:  20180217T191000_maiac_la_0.hdf
Granule ID:  20180217T191000_maiac_la_0.hdf
Granule ID:  20180217T191000_

Granule ID:  20180221T035500_maiac_tpe_0.hdf
Granule ID:  20180221T022000_maiac_tpe_0.hdf
Granule ID:  20180221T035500_maiac_tpe_0.hdf
Granule ID:  20180221T022000_maiac_tpe_0.hdf
Granule ID:  20180221T035500_maiac_tpe_0.hdf
Granule ID:  20180221T071500_maiac_dl_0.hdf
Granule ID:  20180221T071500_maiac_dl_0.hdf
Granule ID:  20180221T071500_maiac_dl_0.hdf
Granule ID:  20180221T071500_maiac_dl_0.hdf
Granule ID:  20180221T071500_maiac_dl_0.hdf
Granule ID:  20180221T071500_maiac_dl_0.hdf
Granule ID:  20180221T071500_maiac_dl_0.hdf
Granule ID:  20180221T071500_maiac_dl_0.hdf
Granule ID:  20180221T071500_maiac_dl_0.hdf
Granule ID:  20180221T071500_maiac_dl_0.hdf
Granule ID:  20180221T071500_maiac_dl_0.hdf
Granule ID:  20180222T192500_maiac_la_0.hdf
Granule ID:  20180222T192500_maiac_la_0.hdf
Granule ID:  20180222T192500_maiac_la_0.hdf
Granule ID:  20180222T192500_maiac_la_0.hdf
Granule ID:  20180222T192500_maiac_la_0.hdf
Granule ID:  20180222T192500_maiac_la_0.hdf
Granule ID:  20180222T19250

...

Granule ID:  20180703T033500_maiac_tpe_0.hdf
Granule ID:  20180703T033500_maiac_tpe_1.hdf
Granule ID:  20190703T030000_maiac_tpe_0.hdf
Granule ID:  20190703T030000_maiac_tpe_1.hdf
Granule ID:  20180703T065000_maiac_dl_0.hdf
Granule ID:  20190703T062000_maiac_dl_0.hdf
Granule ID:  20180703T065000_maiac_dl_0.hdf
Granule ID:  20190703T062000_maiac_dl_0.hdf
Granule ID:  20180703T065000_maiac_dl_0.hdf
Granule ID:  20190703T062000_maiac_dl_0.hdf
Granule ID:  20180703T065000_maiac_dl_0.hdf
Granule ID:  20190703T062000_maiac_dl_0.hdf
Granule ID:  20180703T065000_maiac_dl_0.hdf
Granule ID:  20190703T062000_maiac_dl_0.hdf
Granule ID:  20180703T065000_maiac_dl_0.hdf
Granule ID:  20190703T062000_maiac_dl_0.hdf
Granule ID:  20180703T065000_maiac_dl_0.hdf
Granule ID:  20190703T062000_maiac_dl_0.hdf
Granule ID:  20180703T065000_maiac_dl_0.hdf
Granule ID:  20190703T062000_maiac_dl_0.hdf
Granule ID:  20180704T204000_maiac_la_0.hdf
Granule ID:  20190704T200500_maiac_la_0.hdf
Granule ID:  20180704T204000

Granule ID:  20190706T033000_maiac_tpe_0.hdf
Granule ID:  20190706T033000_maiac_tpe_1.hdf
Granule ID:  20180706T054500_maiac_dl_0.hdf
Granule ID:  20190706T065000_maiac_dl_0.hdf
Granule ID:  20180706T054500_maiac_dl_0.hdf
Granule ID:  20190706T065000_maiac_dl_0.hdf
Granule ID:  20180706T054500_maiac_dl_0.hdf
Granule ID:  20190706T065000_maiac_dl_0.hdf
Granule ID:  20180706T054500_maiac_dl_0.hdf
Granule ID:  20190706T065000_maiac_dl_0.hdf
Granule ID:  20180706T054500_maiac_dl_0.hdf
Granule ID:  20190706T065000_maiac_dl_0.hdf
Granule ID:  20180706T054500_maiac_dl_0.hdf
Granule ID:  20190706T065000_maiac_dl_0.hdf
Granule ID:  20180706T054500_maiac_dl_0.hdf
Granule ID:  20190706T065000_maiac_dl_0.hdf
Granule ID:  20180707T193500_maiac_la_0.hdf
Granule ID:  20190707T203500_maiac_la_0.hdf
Granule ID:  20180707T193500_maiac_la_0.hdf
Granule ID:  20190707T203500_maiac_la_0.hdf
Granule ID:  20180707T193500_maiac_la_0.hdf
Granule ID:  20190707T203500_maiac_la_0.hdf
Granule ID:  20180707T193500_m

Granule ID:  20190709T054000_maiac_dl_0.hdf
Granule ID:  20180709T061500_maiac_dl_0.hdf
Granule ID:  20190709T054000_maiac_dl_0.hdf
Granule ID:  20180709T061500_maiac_dl_0.hdf
Granule ID:  20190709T054000_maiac_dl_0.hdf
Granule ID:  20180709T061500_maiac_dl_0.hdf
Granule ID:  20190709T054000_maiac_dl_0.hdf
Granule ID:  20180709T061500_maiac_dl_0.hdf
Granule ID:  20190709T054000_maiac_dl_0.hdf
Granule ID:  20180709T061500_maiac_dl_0.hdf
Granule ID:  20190709T054000_maiac_dl_0.hdf
Granule ID:  20180710T200500_maiac_la_0.hdf
Granule ID:  20190710T193000_maiac_la_0.hdf
Granule ID:  20180710T200500_maiac_la_0.hdf
Granule ID:  20190710T193000_maiac_la_0.hdf
Granule ID:  20180710T200500_maiac_la_0.hdf
Granule ID:  20190710T193000_maiac_la_0.hdf
Granule ID:  20180710T200500_maiac_la_0.hdf
Granule ID:  20190710T193000_maiac_la_0.hdf
Granule ID:  20180710T200500_maiac_la_0.hdf
Granule ID:  20190710T193000_maiac_la_0.hdf
Granule ID:  20180710T200500_maiac_la_0.hdf
Granule ID:  20190710T193000_mai

Granule ID:  20190711T070500_maiac_dl_0.hdf
Granule ID:  20180711T060000_maiac_dl_0.hdf
Granule ID:  20190711T070500_maiac_dl_0.hdf
Granule ID:  20180711T060000_maiac_dl_0.hdf
Granule ID:  20190711T070500_maiac_dl_0.hdf
Granule ID:  20180711T060000_maiac_dl_0.hdf
Granule ID:  20190711T070500_maiac_dl_0.hdf
Granule ID:  20180711T060000_maiac_dl_0.hdf
Granule ID:  20190711T070500_maiac_dl_0.hdf
Granule ID:  20180711T060000_maiac_dl_0.hdf
Granule ID:  20190711T070500_maiac_dl_0.hdf
Granule ID:  20180711T060000_maiac_dl_0.hdf
Granule ID:  20190711T070500_maiac_dl_0.hdf
Granule ID:  20180711T060000_maiac_dl_0.hdf
Granule ID:  20190711T070500_maiac_dl_0.hdf
Granule ID:  20180711T060000_maiac_dl_0.hdf
Granule ID:  20190711T070500_maiac_dl_0.hdf
Granule ID:  20180711T060000_maiac_dl_0.hdf
Granule ID:  20190711T070500_maiac_dl_0.hdf
Granule ID:  20180711T060000_maiac_dl_0.hdf
Granule ID:  20190711T070500_maiac_dl_0.hdf
Granule ID:  20180711T060000_maiac_dl_0.hdf
Granule ID:  20190711T070500_mai

Granule ID:  20190713T034000_maiac_tpe_0.hdf
Granule ID:  20190713T033500_maiac_tpe_0.hdf
Granule ID:  20180713T023000_maiac_tpe_0.hdf
Granule ID:  20180713T041000_maiac_tpe_0.hdf
Granule ID:  20190713T034000_maiac_tpe_0.hdf
Granule ID:  20190713T033500_maiac_tpe_0.hdf
Granule ID:  20180713T055000_maiac_dl_0.hdf
Granule ID:  20190713T065500_maiac_dl_0.hdf
Granule ID:  20180713T055000_maiac_dl_0.hdf
Granule ID:  20190713T065500_maiac_dl_0.hdf
Granule ID:  20180713T055000_maiac_dl_0.hdf
Granule ID:  20190713T065500_maiac_dl_0.hdf
Granule ID:  20180713T055000_maiac_dl_0.hdf
Granule ID:  20190713T065500_maiac_dl_0.hdf
Granule ID:  20180713T055000_maiac_dl_0.hdf
Granule ID:  20190713T065500_maiac_dl_0.hdf
Granule ID:  20180713T055000_maiac_dl_0.hdf
Granule ID:  20190713T065500_maiac_dl_0.hdf
Granule ID:  20180713T055000_maiac_dl_0.hdf
Granule ID:  20190713T065500_maiac_dl_0.hdf
Granule ID:  20180713T055000_maiac_dl_0.hdf
Granule ID:  20190713T065500_maiac_dl_0.hdf
Granule ID:  20180713T0550

In [29]:
def baseline_model():
    model = Sequential()
    model.add(Dense(1, input_dim=57600, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

print(features.shape)
print(labels.shape)

estimator = KerasRegressor(build_fn=baseline_model, epochs=10, batch_size=5, verbose=0)
results = cross_val_score(estimator, features, labels)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

(9999, 57600)
(9999,)


C:\Users\ZIXIAN~1\AppData\Local\Temp/ipykernel_18796/2622334013.py:23: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  estimator = KerasRegressor(build_fn=baseline_model, epochs=10, batch_size=5, verbose=0)


Baseline: -295384900.70 (590002344.83) MSE
